# som with scikit-learn manner
sksom is scikit-learn api for SimpleSOM. This class has method below  
* fit
* predict
* predict_proba

and attributes labels\_ too.

In [ ]:
%matplotlib inline
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

Using TensorFlow backend.
/home/admin/miniconda3/envs/da02/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
#/home/admin/github/wordroid.sblo.jp/lib
import os, sys
sys.path.append('/home/admin/github/wordroid.sblo.jp/lib')
from som import som

In [ ]:
import random

import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import preprocessing
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
iris = datasets.load_iris()
print(dir(iris))
X = iris.data
print(iris.target_names)
X_sc = preprocessing.scale(X)

In [ ]:
X_sc[:3]

In [ ]:
iris.target

### prepare initial landmarks
You can obtain initial landmarks using SimpleSOM.

In [ ]:
sobj_makeK = som.SimpleSOM((20, 30), initialization_func='linear')
#sobj_makeK = som.SimpleSOM((20, 30))
sobj_makeK

In [ ]:
sobj_makeK._initialize(X_sc)
sobj_makeK.K.shape

In [ ]:
img = som.conv2img(sobj_makeK.K, (20, 30))
plt.figure(figsize=(10, 10))
plt.imshow(img)

### sksom

In [ ]:
'''
argument init_K must be needed
'''
sobj = som.sksom((20, 30), init_K=sobj_makeK.K, it=50, verbose=2, early_stopping=False)
#sobj = som.sksom((20, 30), init_K=sobj_makeK.K, r=1.5, it=1500, verbose=2)
sobj

In [ ]:
img = som.conv2img(sobj.landmarks_, (20, 30))
plt.figure(figsize=(10, 10))
plt.imshow(img)

## fit
r decreases linearly to one during training, if r=None

In [ ]:
sobj.fit(X_sc)

In [ ]:
img = som.conv2img(sobj.landmarks_, (20, 30))
plt.figure(figsize=(10, 10))
plt.imshow(img)

In [ ]:
img = som.conv2img(sobj.landmarks_, (20, 30))
plt.figure(figsize=(10, 10))
plt.imshow(img)

for i, m in enumerate(sobj.predict(X_sc)):
    b, a = divmod(m, sobj.kshape[1])
    if iris.target_names[iris.target[i]] == 'versicolor':
        plt.text(a, b, 'versicolor', ha='center', va='center',
               bbox=dict(facecolor='lightblue', alpha=0.5, lw=0))
    elif iris.target_names[iris.target[i]] == 'virginica':
        plt.text(a, b, 'virginica', ha='center', va='center',
               bbox=dict(facecolor='pink', alpha=0.5, lw=0))
    else:
        plt.text(a, b, 'setosa', ha='center', va='center',
               bbox=dict(facecolor='white', alpha=0.5, lw=0))

In [ ]:
df = pd.DataFrame(sobj.landmarks_)
sns.pairplot(df, markers='.')

In [ ]:
df = pd.DataFrame(X_sc)
sns.pairplot(df, markers='.')

In [ ]:
df1= pd.DataFrame(sobj.landmarks_)
df1['cls'] = 'K'
df1.head()
df2 = pd.DataFrame(X_sc)
df2['cls'] = 'X'
df2.head()
df = pd.concat([df1, df2], axis=0)
df.head()
df.shape
sns.pairplot(df, markers=['.', 's'], hue='cls', plot_kws={'alpha': 0.5})

### further train

In [ ]:
'''
r: 1.5 -> 1.0
'''
sobj.it = 500
sobj.r = (1.5, 1.2)
sobj.verbose = 2
sobj.fit(X_sc)

In [ ]:
lw = 2
plt.plot(np.arange(len(sobj.meanDist)), sobj.meanDist, label="mean distance to closest landmark",
         color="darkorange", lw=lw)
plt.legend(loc="best")

In [ ]:
img = som.conv2img(sobj.landmarks_, (20, 30))
plt.figure(figsize=(10, 10))
plt.imshow(img)

In [ ]:
img = som.conv2img(sobj.landmarks_, (20, 30))
plt.figure(figsize=(10, 10))
plt.imshow(img)

for i, m in enumerate(sobj.predict(X_sc)):
    b, a = divmod(m, sobj.kshape[1])
    if iris.target_names[iris.target[i]] == 'versicolor':
        plt.text(a, b, 'versicolor', ha='center', va='center',
               bbox=dict(facecolor='lightblue', alpha=0.5, lw=0))
    elif iris.target_names[iris.target[i]] == 'virginica':
        plt.text(a, b, 'virginica', ha='center', va='center',
               bbox=dict(facecolor='pink', alpha=0.5, lw=0))
    else:
        plt.text(a, b, 'setosa', ha='center', va='center',
               bbox=dict(facecolor='white', alpha=0.5, lw=0))

In [ ]:
df = pd.DataFrame(sobj.landmarks_)
sns.pairplot(df, markers='.')

In [ ]:
df = pd.DataFrame(X_sc)
sns.pairplot(df, markers='.')

In [ ]:
df1= pd.DataFrame(sobj.landmarks_)
df1['cls'] = 'K'
df1.head()
df2 = pd.DataFrame(X_sc)
df2['cls'] = 'X'
df2.head()
df = pd.concat([df1, df2], axis=0)
df.head()
df.shape
sns.pairplot(df, markers=['.', 's'], hue='cls', plot_kws={'alpha': 0.5})

## labels_

In [ ]:
sobj.labels_

## predict
predict returns the nearest labels of input

In [ ]:
sobj.predict(sobj.landmarks_[:30])

In [ ]:
sobj.predict(X_sc)

## predict_proba
predict_proba returns probability array of shape = [n_query, n_landmarks]

In [ ]:
'''
evaluate by gaussian kernel with gamma
'''
print(sobj.som.gamma)
sobj.predict_proba(X_sc)[:10,:5]

In [ ]:
sobj.predict_proba(sobj.landmarks_[:15])[:,:5]